In [ ]:
seq_file = r'./datasets/searches\DIAUmpire\DIAUmpire_RA957_seqs.tsv'
save_as = './model/RA957_HLA_no_pretrain_model.pth'
pretrained_model = None

In [1]:
import pandas as pd
pep_df = pd.read_csv(seq_file)
pep_df['nAA'] = pep_df.sequence.str.len()
pep_df['HLA'] = 1
pep_df

,sequence,nAA,HLA
0,ETQGQQPPQR,10,1
1,ESAPEGQAQQR,11,1
2,NRNDQEATL,9,1
3,EHVKEVQQL,9,1
4,NHHLQETSF,9,1
...,...,...,...
16747,EHMELVSRL,9,1
16748,VTPQIDSSRI,10,1
16749,MPVSELTDKL,10,1
16750,IPISHIDDVL,10,1


In [2]:
def split_to_two_dfs(df, ratio=0.8):
    train_df = df.sample(frac=ratio, replace=False)
    test_df = df.drop(train_df.index)
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

pos_train_df, pos_test_df = split_to_two_dfs(pep_df, 0.8)

In [6]:
from peptdeep_hla.utils import get_random_sequences

def concat_neg_df(pos_df, prot_df):
    df_list = [pos_df]
    for nAA, group_df in pos_df.groupby('nAA'):
        rnd_seqs = get_random_sequences(
            prot_df, 
            n=len(group_df),
            pep_len = nAA
        )
        df_list.append(pd.DataFrame(
            dict(sequence=rnd_seqs,nAA=nAA,HLA=0)
        ))
    return pd.concat(df_list).reset_index(drop=True)

,sequence,nAA,HLA
0,RVIEEAKTAFL,11,1
1,TDAWGIKVERVEIK,14,1
2,IPNPQERTLTL,11,1
3,DTAGTEQFTAMR,12,1
4,YYDPMISKL,9,1
...,...,...,...
3161,AEAVSHIQSSGPRR,14,0
3162,APLYKVRFPKVQLQ,14,0
3163,RRVLLTQAKNQFAA,14,0
3164,IEYKDFKLIYRQYA,14,0


In [7]:
def evaluate(model, test_df, prob=0.5):
    model.predict(test_df)
    test_df[test_df.HLA_prob_pred>prob]['HLA'].mean(), test_df[test_df.HLA_prob_pred>prob]['HLA'].sum()/len(test_df)*2

## PyTorch Classifier

In [10]:
from peptdeep_hla.HLA_class_I import HLA_Class_I_Classifier

## Build, train and predict

In [11]:
model = HLA_Class_I_Classifier(
    fasta_files=['/User/Feng/fasta/uniprot_human_reviewed_20210309.fasta']
)
if pretrained_model:
    model.load(pretrained_model)
model.get_parameter_num()

1669697

In [12]:
model.train(pos_train_df, epoch=100, warmup_epoch=20, verbose=True)

[Training] Epoch=1, lr=5e-06, loss=0.6939418395360311
[Training] Epoch=2, lr=1e-05, loss=0.6938882946968079
[Training] Epoch=3, lr=1.5e-05, loss=0.6937669595082601
[Training] Epoch=4, lr=2e-05, loss=0.6935871243476868
[Training] Epoch=5, lr=2.5e-05, loss=0.6933180014292399
[Training] Epoch=6, lr=3e-05, loss=0.6928277373313904
[Training] Epoch=7, lr=3.5e-05, loss=0.6917666554450989
[Training] Epoch=8, lr=4e-05, loss=0.6881247838338216
[Training] Epoch=9, lr=4.5e-05, loss=0.6750718553860983
[Training] Epoch=10, lr=5e-05, loss=0.6374742309252421
[Training] Epoch=11, lr=5.500000000000001e-05, loss=0.5830994645754496
[Training] Epoch=12, lr=6e-05, loss=0.49845558603604634
[Training] Epoch=13, lr=6.500000000000001e-05, loss=0.45616323749224347
[Training] Epoch=14, lr=7e-05, loss=0.4299861987431844
[Training] Epoch=15, lr=7.500000000000001e-05, loss=0.40282535354296367
[Training] Epoch=16, lr=8e-05, loss=0.3884335776170095
[Training] Epoch=17, lr=8.5e-05, loss=0.37744767864545187
[Training] E

In [13]:
model.save(save_as)

In [14]:
test_df = concat_neg_df(pos_test_df, model.protein_df)
model.predict(test_df)

,sequence,nAA,HLA,HLA_prob_pred
0,ETILASSGTDR,11,1,0.988257
1,RLAQHITYV,9,1,0.963363
2,DTITETDLR,9,1,0.984478
3,HHVLQNDVL,9,1,0.985039
4,KLDQDLNEV,9,1,0.936592
...,...,...,...,...
6421,KLRRNGLEPVTYHR,14,0,0.025646
6422,AFDGDGDRNMILGK,14,0,0.420663
6423,QIWFQNRRSRLLLQ,14,0,0.004337
6424,FSLMGDCARWELSE,14,0,0.003082


In [15]:
prob=0.7
print("Precision =", test_df[test_df.HLA_prob_pred>prob]['HLA'].mean())
print("Recall =", test_df[test_df.HLA_prob_pred>prob]['HLA'].sum()/len(test_df)*2)
print("False Positive Rate =", 1-(1-test_df[test_df.HLA_prob_pred<prob]['HLA']).sum()/len(test_df)*2)

Precision = 0.9396175779939617
Recall = 0.87177093059446
False Positive Rate = 0.05602240896358546


# Apply the model for all protein sequences

The biggest computational problem is that non-specific digestion and removing redundunt peptide sequences. Here we used longest common prefix (LCP)-based algorithm in `alphapeptdeep` (Feng) to fast generate all non-redundunt substrings. Digestion for 20K proteins could be done in seconds using LCP algorithm.

# Get all sequences with >prob (0.7) predicted probabilities

In [22]:
hla_df = model.predict_from_proteins(prob_threshold=prob)
hla_df

,start_pos,end_pos,nAA,HLA_prob_pred,sequence
5,4736571,4736579,8,0.756716,QAFHSVAF
29,4736595,4736603,8,0.815665,RLKDLLGR
99,4736651,4736659,8,0.817420,FSSRAVAR
131,4736620,4736628,8,0.817954,GPEPGQAR
144,4736436,4736444,8,0.832021,TTYTHCRL
...,...,...,...,...,...
73180109,2044507,2044521,14,0.740652,MHQQQELQREGQRL
73180133,6918665,6918679,14,0.955907,SVVPESDTSERSSM
73180150,626345,626359,14,0.962231,EPSNEPSPALFHKK
73180161,2044502,2044516,14,0.725886,ELDASMHQQQELQR


In [23]:
hla_df.to_csv('Predicted_RA957_no_pretrain_HLA.tsv',index=False, sep="\t")